## Imports
---

In [19]:
import torch

torch.cuda.is_available()

False

In [1]:
from model.model import CNet2D
from model.dataloader import NearlabDatasetLoader, NinaProDatasetLoader
from model.utils import list_files
import scipy.io
import numpy as np
import pandas as pd

In [2]:
path_ninapro = "/Users/dennisschielke/Desktop/Uni/Bachelor_Thesis/src/data/ninapro/DB2/person1"

data = NinaProDatasetLoader(path_ninapro, 1, 1) 

normal = data.load_few_shot_learning_data()

(5238693, 12)
(5238693,)


ValueError: Length of values (5238692) does not match length of index (5238693)